# Fourier analysis tests
In this notebook I use simple analytical signals to test the effect of the following list on the fourier spectra (amplitude and phase):
1. effect of sampling frequency
2. effect of contant or linearly increasing term in signal
3. effect of different sine signal at different freq (in combination with sampling frequency)
4. effect of nodal phase corrections and nodel modulation amplitude corrections which are time dependent
5. effect of averaging multiple signals

Most simple version of tidal signal is:  
$$ U_{\mathrm{M_2}} = A_{M_2} cos(\omega_{M_2} t)  $$
where we use $M_2$ as this is the dominant tidal constituent. $T_{M_2} = 12.44$ h  
we use $A_{M_2} = 1$ m/s 

In [ ]:
# import needed packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from scipy.fft import fft, ifft, fftfreq
from datetime import datetime, timedelta
from scipy.stats import circmean # for circular mean
from utide import solve
plt.style.use('../python_style_Meike.mplstyle')
rad2deg = 180 / np.pi


In [ ]:
# define function
def U_tide(t,A,omega, phi=0):
    return A * np.cos( omega * t - phi)



def Amplitude(f_signal):
    norm = 2/ f_signal.size 
    end = int(0.5 * f_signal.size)
    return norm *  np.abs(f_signal)[:end]

def Phase(f_signal, norm = 1):
    end = int(0.5 * f_signal.size)
    return norm * np.angle(f_signal)[:end]

def interpolated_value(x,x1,x2,f1,f2):
    """
    calculated interpolated value f at location x given it is 
    located between x1 with value f1 and x2 with value f2
    """
    return (np.abs(x - x1) * f1 + np.abs(x - x2) * f2) / np.abs(x1 - x2)

TM2 = 360/28.9841042
omega_M2 = 2*np.pi / TM2
AM2 = 1

print(TM2)

## Effect of sampling frequency

In [ ]:
# effect of sampling frequency
tmin = 0 #h
tmax = 100 #h
tmax_sample = 33
dt = 0.1 #h
dt_sample =  1
t = np.arange(tmin, tmax, dt)
t_sample = np.arange(tmin,tmax_sample, dt_sample)
fig, axs = plt.subplots(3,1,figsize=(30,20))

#define velocity signals
U = U_tide(t, AM2, omega_M2)
U_sample = U_tide(t_sample, AM2, omega_M2)
axs[0].plot(t, U, '-s',color='navy')
axs[0].plot(t_sample, U_sample, 'o',color='firebrick')
axs[0].set_xlabel('time [h]')
axs[0].set_ylabel('velocity [m/s]')

#calculate fourier transforms and frequencies
f_U = fft(U)
f_U_sample = fft(U_sample)
freq_U = fftfreq(U.size, d=dt)[0:int(0.5 * U.size)]
freq_sample = fftfreq(U_sample.size, d=dt_sample)[0:int(0.5 * U_sample.size)]

axs[1].plot(1/ freq_U, Amplitude(f_U),'-s',color='navy')
axs[1].plot(1 / freq_sample, Amplitude(f_U_sample),'--o',color='firebrick')
axs[1].set_xlim(0,20)
axs[1].legend([f'T = {tmax} h, dt = {dt} h',f'T = {tmax_sample} h, dt = {dt_sample} h'])
axs[1].axvline(TM2, color='grey',zorder=-5)
axs[1].set_xlabel('period [h]')
axs[1].set_ylabel('Amplitude [m/s]')

axs[2].plot(1 / freq_U, Phase(f_U, rad2deg),'-s',color='navy')
axs[2].plot(1 / freq_sample, Phase(f_U_sample, rad2deg),'--o',color='firebrick')
axs[2].set_xlim(0,20)
axs[2].axvline(TM2, color='grey',zorder=-5)
axs[2].set_xlabel('period [h]')
axs[2].set_ylabel('phase [$^\\circ$]')






effect of sampling frequency:   
clearly there is a big effect of sampling period an sampling frequency used. To compare signals one has to make sure to use the same sampling period. However I think it would be better to try to calculate the signal at the desired frequency (omega_M2) as the signal of M2 is now disctributed on the frequencies directly below and above omega_M2 making it hard to read the signal. one way to to this is to reinterpolate the original data to this sampling frequency but I would expect that there is some neat fourier trick to do this.

In [ ]:
1/freq_sample[5]
Amplitude(f_U_sample)[5]

In [ ]:
latmean=54
ttest_U= np.arange(np.datetime64("2015-01-01"), np.datetime64("20015-01-06"), np.timedelta64(6, "m"))[0:1000]
ttest_Usample=np.arange(np.datetime64("2015-01-01"), np.datetime64("20015-01-06"), np.timedelta64(1, "h"))[0:U_sample.size]
print(U.size)
print(ttest.size)
coef_U = solve(
    ttest_U,
    U,
    lat=latmean,
    nodal=False,
    trend=False,
    method="ols",
    conf_int="linear",
    Rayleigh_min=0.95)

coef_Usample = solve(
    ttest_Usample,
    U_sample,
    lat=latmean,
    nodal=False,
    trend=False,
    method="ols",
    conf_int="linear",
    Rayleigh_min=0.95)



In [ ]:
print(coef_U.A)
print(coef_U.name)
print(coef_Usample.A)
print(coef_Usample.name)
fig,ax=plt.subplots()
ax.plot(coef_U.A,'o',color='navy')
ax.plot(coef_Usample.A,'s',color='firebrick')